## Entrega 6
### Ignacio Ziccardi
### Realizar una tabla por cada SNR, que describa el sesgo y la varianza de cada estimador para cada ventana analizada. Recuerde incluir las ventanas rectangular (sin ventana), flattop y blackmanharris y otras que considere (considero Hamming). 

$x(k)=a_0⋅sen(Ω_1⋅k)+n_a(n)$<br>
siendo<br>
$Ω_1=Ω_0+f_r⋅2πN$<br>
$Ω_0=\frac{π}{2}$<br>
siendo la variable aleatoria definida por la siguiente distribución de probabilidad<br>
$f_r∼U(−2,2)$<br>
$na∼N(0,σ^2)$<br>
Diseñe los siguientes estimadores,  de amplitud $a_1$<br>
$a^i_1=|X^i_w(Ω_0)|=|F{x(n)⋅w_i(n)}|$<br>
para la  $i$-ésima realización y la  $w$-ésima ventana (ver detalles debajo). y de frecuencia $Ω_1$<br>
$Ω^i_1=arg max{|X^i_w(Ω)|}$


In [46]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import random as random
#%%
Fo = 1.0
Fs = 1000.0 # frecuencia de muestreo (Hz)
N = 1000   # cantidad de muestras
ts = 1/Fs # tiempo de muestreo
df = Fs/N # resolución espectral
Ac =2**0.5 #Amplitud 
DC = 0 #Valor Contínua
tita = 0 #Defasaje
#----------------------------------------Estos 3 se repetirán cuando se cambie la SNR
SNR = 3 # SNR en dB
sigma = (10**(-SNR/10)) #Varianza
desvio = sigma**0.5 #Desvío Estándar
#----------------------------------------
Omega_0 = Fs/4
R = 200

In [47]:
#%% Funciones
def my_sin_gen( vmax, dc, fo, ph, nn, fs): 
    tt = np.arange(0,nn*1/fs,1/fs) # grilla de sampleo temporal
    xx = vmax*np.sin(2*np.pi*fo*tt + ph) + dc #Senoidal
    return [tt, xx]

### Genero las señales requeridas con sus funciones de probabilidad correspondientes

In [48]:
#Genero señal aleatoria para el ruido
na=np.random.normal(0, desvio, (R,N))
#Genero la variable Omega según la distribución de fr
fr=np.random.uniform(-2,2,R).reshape((R,1))
Omega_1 = Omega_0 + fr*df

tt, s = my_sin_gen(vmax = Ac, dc = DC, fo = Fo*Omega_1, ph=tita, nn = N, fs =Fs )

ss= s + na

### Aplico la Ventana Requerida y Obtengo las transformadas para obtener los estimadores de cada ventana
Nota: No se plotea el gráfico con las 200 trasnformadas porque se corroboró con antelación para que no queden muchos gráficos en la entrega

**Ventana Recctangular**

In [49]:
wi = sig.windows.get_window('boxcar', len(tt))

X_R = ss * wi

TFF_X=np.fft.fft(X_R, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_3dB = Esperanza_a - Ac
V_a_3dB = sum((a_estimador-Esperanza_a)**2)/200



**Ventana Flattop**

In [50]:
wi = sig.windows.get_window('flattop', len(tt))

X_FL = ss * wi

TFF_X=np.fft.fft(X_FL, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_FL_3dB = Esperanza_a - Ac
V_a_FL_3dB = sum((a_estimador-Esperanza_a)**2)/200

**Ventana Blackmanharris**

In [51]:
wi = sig.windows.get_window('blackmanharris', len(tt))

X_FL = ss * wi

TFF_X=np.fft.fft(X_FL, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_BL_3dB = Esperanza_a - Ac
V_a_BL_3dB = sum((a_estimador-Esperanza_a)**2)/200

**Ventana Hamming**

In [52]:
wi = sig.windows.get_window('hamming', len(tt))

X_FL = ss * wi

TFF_X=np.fft.fft(X_FL, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_HG_3dB = Esperanza_a - Ac
V_a_HG_3dB = sum((a_estimador-Esperanza_a)**2)/200

### Ahora se repetirá el proceso anterior pero cambiando la SNR a 10dB

In [53]:
SNR = 10 # SNR en dB
sigma = (10**(-SNR/10)) #Varianza
desvio = sigma**0.5 #Desvío Estándar

In [54]:
#Genero señal aleatoria para el ruido
na=np.random.normal(0, desvio, (R,N))
#Genero la variable Omega según la distribución de fr
fr=np.random.uniform(-2,2,R).reshape((R,1))
Omega_1 = Omega_0 + fr*df

tt, s = my_sin_gen(vmax = Ac, dc = DC, fo = Fo*Omega_1, ph=tita, nn = N, fs =Fs )

ss= s + na

**Ventana Rectangular**

In [55]:
wi = sig.windows.get_window('boxcar', len(tt))

X_R = ss * wi

TFF_X=np.fft.fft(X_R, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_10dB = Esperanza_a - Ac
V_a_10dB = sum((a_estimador-Esperanza_a)**2)/200

**Ventana Flattop**

In [56]:
wi = sig.windows.get_window('flattop', len(tt))

X_FL = ss * wi

TFF_X=np.fft.fft(X_FL, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_FL_10dB = Esperanza_a - Ac
V_a_FL_10dB = sum((a_estimador-Esperanza_a)**2)/200

**Ventana Blackmanharris**

In [57]:
wi = sig.windows.get_window('blackmanharris', len(tt))

X_FL = ss * wi

TFF_X=np.fft.fft(X_FL, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_BL_10dB = Esperanza_a - Ac
V_a_BL_10dB = sum((a_estimador-Esperanza_a)**2)/200

**Ventana Hamming**

In [58]:
wi = sig.windows.get_window('hamming', len(tt))

X_FL = ss * wi

TFF_X=np.fft.fft(X_FL, axis = -1)/len(tt)#Espectro de la señal cuantizada
mod_X=np.abs(TFF_X)
mod_X_LOG=10*np.log10(2*np.abs(TFF_X)**2)

#st = int (N/4) - 10
#fin = int (N/4) + 10

#Se comentó el ploteo para verificar 
#plt.figure("DFT Senoidales",figsize=(15, 7))
#plt.title('DFT Senoidales')
#plt.plot(mod_X[0,:])
#plt.plot(mod_X_LOG[:,st:fin].transpose())
#plt.ylabel("DFT Amplitude |X(freq)|")
#plt.legend()
#plt.show()

lim = int (N/4)-1 #Columna de Fs/4, el -1 es porque va de 0 a 999

#Me quedo con la columna correspondiente a pi/2 que sería el equivalente de fs/4 = 1000/4 = 250
a_estimador = mod_X[:,lim] # Estimador para ventana rectangular

Omega_estimador = np.argmax(mod_X,axis=1)#Recordar recortar a Nyquist

Esperanza_a = sum(a_estimador/200)

S_a_HG_10dB = Esperanza_a - Ac
V_a_HG_10dB = sum((a_estimador-Esperanza_a)**2)/200

In [59]:
from tabulate import tabulate
print(tabulate([['Rectangular $3dB$', S_a_3dB,V_a_3dB], ['Flat-top $3dB$', S_a_FL_3dB, V_a_FL_3dB],['Blackman $3dB$', S_a_BL_3dB,V_a_BL_3dB],['Hamming $3dB$', S_a_HG_3dB,V_a_HG_3dB],['Rectangular $10dB$', S_a_10dB,V_a_10dB], ['Flat-top $10dB$', S_a_FL_10dB, V_a_FL_10dB],['Blackman $10dB$', S_a_BL_10dB,V_a_BL_10dB],['Hamming $10dB$', S_a_HG_10dB,V_a_HG_10dB]], headers=['Ventana', "$s_a$",'$v_a$']))


Ventana                $s_a$       $v_a$
------------------  --------  ----------
Rectangular $3dB$   -1.16241  0.0547424
Flat-top $3dB$      -1.29099  0.00149856
Blackman $3dB$      -1.26881  0.00813184
Hamming $3dB$       -1.24516  0.020762
Rectangular $10dB$  -1.15653  0.0548395
Flat-top $10dB$     -1.28633  0.00123765
Blackman $10dB$     -1.26169  0.00724313
Hamming $10dB$      -1.24067  0.0199877


# Genero la siguiente Tabla

| Ventana | $s_a$ | $v_a$  |
|--------------|--------------|--------------|
| Rectangular $3dB$ | {S_a_3dB}  | {V_a_3dB}  |
| Flat-top $3dB$    | {{S_a_FL_3dB}}  | {{V_a_FL_3dB}} |
| Blackman $3dB$    | {{S_a_BL_3dB}}  | {{V_a_BL_3dB}}  |
| Hamming $3dB$     | {{S_a_HG_3dB}}  | {{V_a_HG_3dB}}  |
| Rectangular $10dB$| {{S_a_10dB}}  |  {{V_a_10dB}} |
| Flat-top $10dB$	| {{S_a_FL_10dB}}  | {{V_a_FL_10dB}}  |
| Blackman $10dB$   | {{S_a_BL_10dB}}  | {{V_a_BL_10dB}}  |
| Hamming $10dB$    | {{S_a_HG_10dB}}  | {{V_a_HG_10dB}}  |